In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import csv
import os
import random
from tqdm import tqdm
from tqdm.notebook import tnrange

import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
import time

import GeoImageDataset   # USE YOUR OWN
import GeoGuessrLoss

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
torch.cuda.get_device_name(device=None)


from GeoGuessrDataset import GeoGuessrDataset
%matplotlib inline

# Enable autoreloading of imported modules.
%load_ext autoreload
%autoreload 2

cuda:0


Create Dataset and (if needed) CSV-file for the dataset

In [2]:
"""

full_dataloader = None

# OVERWRITE OR DELETE MY "location_data.csv" and "GeoImageDataset.py"
# EXAMPLE WITH MY DATASET. DELETE AND REPLACE WITH YOURS
image_paths = [
    "..\\..\\..\\geogussr1",
    "..\\..\\..\\geogussr2",
    "..\\..\\..\\geogussr3",
    "..\\..\\..\\geogussr4",
    "..\\..\\..\\geogussr5",
    "..\\..\\..\\geogussr6",
    "..\\..\\..\\geogussr7"
]
eval_data_path = "location_data.csv"

CSV_needed = True
# Create full CSV-file
def create_CSV():
    with open(eval_data_path, 'w', newline='') as csvfile:  # Create/open file
        wr = csv.writer(csvfile)
        for path in image_paths:
            for file in os.listdir(os.fsencode(path)):
                filename = os.fsdecode(file)
                wr.writerow([path + "\\" + filename] + [float(x) for x in filename[4:-4].split(",")])

# Define  dataset
if(CSV_needed):
    create_CSV()

full_dataset = GeoImageDataset.GeoImageDataset("", eval_data_path) 
print("Dataset: " ,  len(full_dataset))

full_dataloader = DataLoader(full_dataset, batch_size=8,
                    shuffle=True, num_workers=3)
# ------------------------------------------------------------------
"""

'\n\nfull_dataloader = None\n\n# OVERWRITE OR DELETE MY "location_data.csv" and "GeoImageDataset.py"\n# EXAMPLE WITH MY DATASET. DELETE AND REPLACE WITH YOURS\nimage_paths = [\n    "..\\..\\..\\geogussr1",\n    "..\\..\\..\\geogussr2",\n    "..\\..\\..\\geogussr3",\n    "..\\..\\..\\geogussr4",\n    "..\\..\\..\\geogussr5",\n    "..\\..\\..\\geogussr6",\n    "..\\..\\..\\geogussr7"\n]\neval_data_path = "location_data.csv"\n\nCSV_needed = True\n# Create full CSV-file\ndef create_CSV():\n    with open(eval_data_path, \'w\', newline=\'\') as csvfile:  # Create/open file\n        wr = csv.writer(csvfile)\n        for path in image_paths:\n            for file in os.listdir(os.fsencode(path)):\n                filename = os.fsdecode(file)\n                wr.writerow([path + "\\" + filename] + [float(x) for x in filename[4:-4].split(",")])\n\n# Define  dataset\nif(CSV_needed):\n    create_CSV()\n\nfull_dataset = GeoImageDataset.GeoImageDataset("", eval_data_path) \nprint("Dataset: " ,  len(

In [3]:

# folder with pictures
ROOT_DIR = r'C:\Users\Shadow\Pictures\Geogussr\Projekt'

# dir to csv files
dir = r"C:\Users\Shadow\Documents\sequentialmodel\preprocess"

In [4]:
# Define the data transformation
transform = transforms.Compose([
    transforms.ToTensor(),  # convert images to tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # normalize images
    transforms.Resize((250, 1000))
])

# Load the dataset and split it into training and validation sets (fc_size is generated in the kernel above)
dataset = GeoGuessrDataset(csv_file=dir+'\coordinates3.csv',
                                    root_dir=ROOT_DIR, transform=transform, num_classes=4000, indices=None)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size],generator=torch.Generator(
).manual_seed(42))
print(len(train_dataset))
# Define the dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=6)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=6)

dataloaders = {"train" : train_dataloader, "val": val_dataloader}
dataset_sizes = {"train": train_size, "val" : val_size}

103009


Create the model

In [5]:
from torchvision.models.resnet import resnet50, ResNet50_Weights, resnet18, ResNet18_Weights
# Create model and change the last layer to output only 2 numbers
model = torchvision.models.resnet18(weights=ResNet18_Weights.DEFAULT).cuda()
model.fc = nn.Linear(512, 2).cuda()
model = model.float()

# Loss and optimizer
loss_criterion = GeoGuessrLoss.GeoGuessrLoss()
loss_criterion = loss_criterion.float()
loss_criterion = loss_criterion.cuda()

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

Training

In [6]:
# Define function for training
model_parameter_path = './GeoResNet_1.pth'  # Where the model parameters get loaded from/ saved to
output_frequency = 100  # How often (in batches) the training function should make a print

def train_all_data(epoches = 1):
    # Load model
    model.load_state_dict(torch.load(model_parameter_path))
    train_loss_history = []
    val_loss_history = []
    for epoch in (pbar := tnrange(epoches)):  # loop over the dataset multiple times
        print(f'Epoch {epoch+1}/{epoches}')
        print('-' * 10)# Each epoch has a training and validation phase
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
            
            running_loss = 0.0
            
            
            for idx, batch in enumerate(dataloaders[phase]):
                inputs, labels, gt  = batch["image"], batch["geohash"], batch['gt']
                inputs = inputs.to(device)
                labels = gt.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    # forward + backward + optimize
                    outputs = model(inputs)
                    loss = loss_criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    running_loss += loss.item() * inputs.size(0)
                    
                    """
                    # print statistics
                    if idx % output_frequency == output_frequency-1:
                        # Print time, example label and guesses and average loss
                        named_tuple = time.localtime()
                        time_string = time.strftime("%m/%d/%Y, %H:%M:%S", named_tuple)
                        print(time_string, "\t Label", labels[0], labels[1])
                        print("Output", outputs[0], outputs[1])
                        print(f'[{epoch + 1}, {idx + 1:5d}] loss: {running_loss / output_frequency:.3f}')
                        running_loss = 0.0
                    """
            epoch_loss = running_loss / dataset_sizes[phase]
            print(f'{phase} Loss: {epoch_loss:.4f}')
                
        # Save model
        torch.save(model.state_dict(), model_parameter_path)
    PATH3 = r"C:\Users\Shadow\Documents\markus\losshist\pretrainedresnet18_10epoch_contCNN_stats.tar"
    torch.save({
    'train_loss_history' : train_loss_history,
    'val_loss_history' : val_loss_history
    }, PATH3)
    torch.save(model.state_dict(), model_parameter_path)
    print('Finished Training')
    return

In [7]:
epoques = 5
train_all_data(epoques)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/5
----------
train Loss: 1353.4711
val Loss: 2526.3673
Epoch 2/5
----------
train Loss: 1274.9240
val Loss: 2401.7265
Epoch 3/5
----------
train Loss: 1204.8605
val Loss: 2384.6137
Epoch 4/5
----------
train Loss: 1143.6093
val Loss: 2452.8270
Epoch 5/5
----------
train Loss: 1088.4556
val Loss: 2531.8559
Finished Training
